## Installations

In [ ]:
!pip install kagglehub --quiet

In [ ]:
!pip install kagglehub[pandas-datasets] --quiet

In [ ]:
!pip install pandas --quiet

In [ ]:
import pandas as pd

## Authenticate with Kaggle

In [ ]:
import os
from google.colab import files
files.upload()  # Upload kaggle.json manually

os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


## Loading the dataset directly as a Pandas DataFrame

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "prachi13/customer-analytics",
    "Train.csv",
)
print(df.head())

   ID Warehouse_block Mode_of_Shipment  Customer_care_calls  Customer_rating  \
0   1               D           Flight                    4                2   
1   2               F           Flight                    4                5   
2   3               A           Flight                    2                2   
3   4               B           Flight                    3                3   
4   5               C           Flight                    2                2   

   Cost_of_the_Product  Prior_purchases Product_importance Gender  \
0                  177                3                low      F   
1                  216                2                low      M   
2                  183                4                low      M   
3                  176                4             medium      M   
4                  184                3             medium      F   

   Discount_offered  Weight_in_gms  Reached.on.Time_Y.N  
0                44           1233            

## Question 1: Which mode of shipment is the most reliable in terms of on-time delivery?

What is the distribution  of shipment modes used?

In [ ]:
on_time_rates = df.groupby('Mode_of_Shipment')['Reached.on.Time_Y.N'].mean().sort_values(ascending=False)
print(on_time_rates)

Mode_of_Shipment
Flight    0.601576
Ship      0.597561
Road      0.588068
Name: Reached.on.Time_Y.N, dtype: float64


In [ ]:
print("Flight is the most reliable mode of on-time delivery")

Flight is the most reliable mode of on-time delivery


It groups data by `Mode_of_Shipment` and calculate the on-time delivery percentage for each. It outputs sorted list showing which mode has the highest success rate.

## Question 2: Does the stated importance of a product influence its likelihood of arriving on time?

In [ ]:
importance_impact = df.groupby('Product_importance')['Reached.on.Time_Y.N'].mean().sort_values(ascending=False)
print(importance_impact)

Product_importance
high      0.649789
low       0.592788
medium    0.590450
Name: Reached.on.Time_Y.N, dtype: float64


It aggregates data by `Product_importance` to find the on-time delivery rate for each category. The result will show whether high-importance products are prioritized and have a better on-time delivery record compared to low-importance ones.

## Question 3: How does the number of customer care calls correlate with the final customer rating?

In [ ]:
rating_vs_calls = df.groupby('Customer_care_calls')['Customer_rating'].mean()
print(rating_vs_calls)

Customer_care_calls
2    3.017241
3    2.959901
4    2.998032
5    2.972938
6    3.083909
7    2.995935
Name: Customer_rating, dtype: float64


It groups shipments by the number of `Customer_care_calls` made and computes the average Customer_rating for each group.

There is no particular trend or correlation detected.

## Question 4: Are heavier products given higher discounts, possibly to offset shipping costs or incentivize purchase?

In [ ]:
correlation = df['Weight_in_gms'].corr(df['Discount_offered'])
print(f"Correlation between Weight and Discount: {correlation}")

Correlation between Weight and Discount: -0.37606671513805257


It computes the Pearson correlation coefficient between product `Weight_in_gms` and `Discount_offered`. The output is a single numerical value between -1 and 1.

**`Moderate Negative Correlation` As the increase in weight happens a moderate decrease in discounts occur!**

## Question 5: Which warehouse block has the highest proportion of delayed shipments?

In [ ]:
late_shipment_rate = 1 - df.groupby('Warehouse_block')['Reached.on.Time_Y.N'].mean()
print(late_shipment_rate.sort_values(ascending=False))

Warehouse_block
A    0.413530
C    0.403164
D    0.402399
F    0.401528
B    0.397709
Name: Reached.on.Time_Y.N, dtype: float64


The code determines the late delivery rate for each `Warehouse_block` ('A', 'B', 'C', 'D', 'F').

Currently Warehouse 'A' has the highest proportions of the delayed shipments.

## Question 6: Does offering a substantial discount (suppose >10%) correlate with a better on-time delivery rate?

In [ ]:
#Creating a binary column
df['High_Discount'] = df['Discount_offered'] > 10

discount_effect = df.groupby('High_Discount')['Reached.on.Time_Y.N'].mean()
print(discount_effect)

High_Discount
False    0.46887
True     1.00000
Name: Reached.on.Time_Y.N, dtype: float64


- For shipments with a discount of 10% or less, only about 46.9% arrived on time.

- For shipments with a discount greater than 10%, 100% arrived on time.

## Question 7: Are customers with a high number of prior purchases more likely to recieve their orders on time?

In [ ]:
loyalty_service_level = df.groupby('Prior_purchases')['Reached.on.Time_Y.N'].mean().sort_index()
print(loyalty_service_level)

Prior_purchases
2     0.625240
3     0.640708
4     0.543387
5     0.498834
6     0.559715
7     0.676471
8     0.648438
10    0.573034
Name: Reached.on.Time_Y.N, dtype: float64


It investigates whether the company prioritizes customer loyalty. It groups the data by the `Prior_purchases` column and calculates the on-time delivery rate for each value. The output will reveal if the delivery success rate increases as the number of prior purchases goes up, indicating better service for loyal customers

## Question 8: Are there significant differences in the average product cost and discounts recieved between male and female customers?

In [ ]:
gender_analysis = df.groupby('Gender').agg(
    Average_Cost=('Cost_of_the_Product', 'mean'),
    Average_Discount=('Discount_offered', 'mean')
)
print(gender_analysis)

        Average_Cost  Average_Discount
Gender                                
F         209.255005         13.562489
M         211.154382         13.180785


It provides a demographic breakdown of purchasing behavior. The .agg() function efficiently calculates both the average product cost and average discount for 'M' and 'F' `Gender` categories in a single table. It shows if any bias exists in gender basis dividend.

## Question 9: How does on-time delivery performance across different product weight cateogories?

In [ ]:
bins = [0, 2000, 5000, float('inf')]
labels = ['Light (<2kg)', 'Medium (2-5kg)', 'Heavy (>5kg)']
df['Weight_Category'] = pd.cut(df['Weight_in_gms'], bins=bins, labels=labels, right=False)

weight_performance = df.groupby('Weight_Category')['Reached.on.Time_Y.N'].mean()
print(weight_performance)

Weight_Category
Light (<2kg)      0.677877
Medium (2-5kg)    0.643709
Heavy (>5kg)      0.426745
Name: Reached.on.Time_Y.N, dtype: float64


/tmp/ipython-input-2948963656.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weight_performance = df.groupby('Weight_Category')['Reached.on.Time_Y.N'].mean()


This query segments products by weight to see if logistics are optimized differently for light versus heavy items. The `pd.cut` function creates distinct weight classes. The output will show the on-time delivery rate for each class, highlighting potential bottlenecks or efficiencies in handling products of different weights.

There is a direct correlation between `Weight_Cateogry` and the `Reached.on.Time_Y.N`. As the weight increases, delays in the on-time delivery starts happening!

## Question 10: Are the most expensive products prioritized for a on-time delievery compared to the least expansive ones?

In [ ]:
df['Cost_Quartile'] = pd.qcut(df['Cost_of_the_Product'], q=4, labels=['Q1_Lowest', 'Q2', 'Q3', 'Q4_Highest'])

value_priority = df.groupby('Cost_Quartile')['Reached.on.Time_Y.N'].mean()
print(value_priority)

Cost_Quartile
Q1_Lowest     0.643402
Q2            0.611534
Q3            0.575465
Q4_Highest    0.555844
Name: Reached.on.Time_Y.N, dtype: float64


/tmp/ipython-input-1661242309.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  value_priority = df.groupby('Cost_Quartile')['Reached.on.Time_Y.N'].mean()


The `pd.qcut` function is used to create four robust cost brackets (quartiles). The result will compare the on-time delivery rates across these brackets, clearly showing if higher-value goods (Q4_Highest) receive preferential treatment in the supply chain.

The results show:
- Lower-cost shipments (Q1_Lowest) have the highest on-time delivery rate (64.3%).

- Highest-cost shipments (Q4_Highest) have the lowest on-time delivery rate (55.6%)

**CONCLUSION: As the monetary value increases, the on-time delivery decreases!**